<a href="https://colab.research.google.com/github/yasickogul/mlProjects/blob/main/commentsEmotionPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


download dataset https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment

In [ ]:
pwd

'/content'

In [ ]:
ls /content/drive/MyDrive/archive/Tweets.csv

/content/drive/MyDrive/archive/Tweets.csv


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/archive/Tweets.csv")


In [ ]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
df = df[['airline_sentiment','text']]

In [ ]:
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [ ]:
import nltk
import string
import re
from nltk.stem.porter import PorterStemmer


In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the 'punkt_tab' resource

from nltk.corpus import stopwords

ps = PorterStemmer()  # Initialize the stemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)  # Remove URLs
    text = nltk.word_tokenize(text)  # Tokenize text
    y = []

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(ps.stem(i))  # Apply stemming

    return " ".join(y)  # Join words back into a sentence

df["text_cleaned"] = df["text"].apply(clean_text)  # Apply function to all tweets
df.head()  # Check the dataset after cleaning

,airline_sentiment,text,text_cleaned
0,neutral,@VirginAmerica What @dhepburn said.,virginamerica dhepburn said
1,positive,@VirginAmerica plus you've added commercials t...,virginamerica plu 've ad commerci experi ... t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...,virginamerica n't today ... must mean need tak...
3,negative,@VirginAmerica it's really aggressive to blast...,virginamerica 's realli aggress blast obnoxi `...
4,negative,@VirginAmerica and it's a really big bad thing...,virginamerica 's realli big bad thing


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer with 3000 features
vectorizer = TfidfVectorizer(max_features=3000)

# Convert cleaned text to TF-IDF vectors
X = vectorizer.fit_transform(df["text_cleaned"]).toarray()

# Convert labels (sentiment) into a NumPy array
Y = df["airline_sentiment"].values



In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Train Naïve Bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predict on test set
y_pred_nb = nb_model.predict(X_test)

# Calculate accuracy
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))


Naïve Bayes Accuracy: 0.7219945355191257


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)

# Calculate accuracy
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))


Random Forest Accuracy: 0.7523907103825137


In [ ]:
df.shape

(14640, 15)

In [ ]:
df.groupby('airline_sentiment').describe()

tweet_id                                            \
                     count          mean           std           min   
airline_sentiment                                                      
negative            9178.0  5.692602e+17  7.572474e+14  5.675900e+17   
neutral             3099.0  5.691841e+17  8.068405e+14  5.675883e+17   
positive            2363.0  5.691006e+17  8.112476e+14  5.676555e+17   

                                                                           \
                            25%           50%           75%           max   
airline_sentiment                                                           
negative           5.686511e+17  5.695355e+17  5.698798e+17  5.703106e+17   
neutral            5.684852e+17  5.693080e+17  5.699305e+17  5.703093e+17   
positive           5.684331e+17  5.691980e+17  5.698788e+17  5.703093e+17   

                  airline_sentiment_confidence            ...  \
                                         count      mean  ...   
airline_sentiment                                         ...   
negative                                9178.0  0.933365  ...   
neutral                                 3099.0  0.823303  ...   
positive                                2363.0  0.872039  ...   

                  negativereason_confidence      retweet_count            \
                                        75%  max         count      mean   
airline_sentiment                                                          
negative                                1.0  1.0        9178.0  0.093375   
neutral                                 0.0  0.0        3099.0  0.060987   
positive                                0.0  0.0        2363.0  0.069403   

                                                       
                        std  min  25%  50%  75%   max  
airline_sentiment                                      
negative           0.792865  0.0  0.0  0.0  0.0  44.0  
neutral            0.658037  0.0  0.0  0.0  0.0  28.0  
positive           0.659914  0.0  0.0  0.0  0.0  22.0  

[3 rows x 32 columns]

In [ ]:
neutral_tweets = df[df['airline_sentiment'] == 'neutral']

# Count the number of unique tweets
unique_neutral_tweets = neutral_tweets['text'].nunique()

print(unique_neutral_tweets)

3067


In [ ]:
import joblib

# Save the trained model (Random Forest in this case)
joblib.dump(rf_model, 'sentiment_model.pkl')  # Changed 'model' to 'rf_model'

['sentiment_model.pkl']

In [ ]:
# Load the saved model
model = joblib.load('sentiment_model.pkl')


In [ ]:
pip install flask joblib


In [ ]:
!pip install pyngrok


In [ ]:
import joblib

# Save the trained model
joblib.dump(model, 'sentiment_model.pkl')


['sentiment_model.pkl']

In [ ]:
from google.colab import files

# Download the saved model to your laptop
files.download('sentiment_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>